In [8]:
import pandas as pd
import time
import os
import csv

from selenium import webdriver
from selenium.webdriver.common.by import By
# from seleniumFunction import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==================== GLOBAL VARIABLES
# Set Chrome options:
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-application-cache")
chrome_options.add_argument("--disable-cache")
chrome_options.add_argument("--disk-cache-size=0")

In [9]:
%%capture
df = pd.read_csv('vietnam_accommodation.csv')

In [10]:
df.url.values[20]

'https://www.agoda.com/partners/partnersearch.aspx?hid=10966'

In [11]:
def url_edited(url):
    try:
        parsed_url = urlparse(url)
        new_path = "/vi-vn" + parsed_url.path
        return urlunparse(parsed_url._replace(path=new_path, query=""))
    except:
        return ""

In [12]:
# import requests
# from bs4 import BeautifulSoup

# def get_link(url):
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"
#     }
#     try:
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()  # Raises an error for bad responses (e.g., 404)
#         soup = BeautifulSoup(response.text, 'html.parser')
#         elem = soup.select_one("a[data-element-name='property-card-content']")
#         return elem['href'] if elem else ""
#     except requests.exceptions.RequestException as e:
#         return ""

# # Example usage
# url = df.url.values[20]
# property_card_url = get_link(url)
# property_card_url

''

In [16]:
def url_edited(url):
    try:
        parsed_url = urlparse(url)
        new_path = "/vi-vn" + parsed_url.path
        return urlunparse(parsed_url._replace(path=new_path, query=""))
    except:
        return ""
        
def get_url(url):
    try:
        driver = webdriver.Chrome(chrome_options)
        driver.get(url)
        elem = driver.find_element(By.CSS_SELECTOR, "a[data-element-name='property-card-content']")
        href = elem.get_attribute('href')
        driver.close()
        return href
    except:
        return ""


In [17]:
get_url('https://www.agoda.com/partners/partnersearch.aspx?hid=10966')

'https://www.agoda.com/pan-pacific-hanoi/hotel/hanoi-vn.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2024-11-21&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=VND&isFreeOccSearch=false&tspTypes=16&los=1&searchrequestid=8378b96f-ec7a-42d9-9875-57655195bcd1'